Weasyl Collaborative Filtering Example
-----------------------------------------

This notebook contains an example of user to user collaborative filtering on weasyl. This is something I did a few years ago successfully, but not in a performant manner: It would take tens of seconds to find recommendations for just one user.

This is an attempt to revisit the problem with more performant python libraries.

Before running this notebook, you'll want to create a .csv file with all the favorites from within postgresql as follows:
```
weasyl=# COPY (SELECT userid, targetid FROM favorite WHERE type='s') TO '/tmp/favorites.csv' DELIMITER ',' CSV HEADER;
COPY 4389453
```
or unzip a favorites.csv.gz that I provide.

You'll also need to install `numpy`, `pandas`, `scipy` and `ipython[notebook]` inside your ve to run this code.

I consulted a few sources for this to see what other people have done. The most useful resource was http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/ which I've tried to modify here to use a sparse scipy matrix.

In [35]:
import time

import numpy as np
import numpy.linalg as npl
import pandas as pd
import scipy.sparse

Last time I did this, we used unary ratings (i.e. every submission was either favorited by a user or not). As such it made sense to use the [Jaccard Index](https://en.wikipedia.org/wiki/Jaccard_index) and treat everything in terms of sets.

However, going forward we want to support favorites, likes, and dislikes. So treat all favorites as score `2` (likes will eventually be 1 and dislikes as -1).

In [2]:
df = pd.read_csv('favorites.csv')
df['rating'] = 2  # Everything is favorites currently
df.head()

,userid,targetid,rating
0,3,23,2
1,3,24,2
2,3,25,2
3,3,29,2
4,10,25,2


Now that we've created a dataframe, we want to create a sparse matrix from it. Since our userids and targetids are not contiguous (e.g. the lowest userid who has favorited anything is 3 and not every user has favorites and many favorites are anonymized when we export the db, etc.), we'll make a mapping of new indices to them.

In [3]:
user_map = {x[1]: x[0] for x in enumerate(df.userid.unique())}
item_map = {x[1]: x[0] for x in enumerate(df.targetid.unique())}

# Show a few items
{k: user_map[k] for k in user_map.keys()[:10]}

{3: 0,
 6: 3784,
 10: 1,
 12: 43,
 13: 1242,
 15: 3,
 17: 2,
 20: 158,
 21: 266,
 131083: 37171}

Now construct our sparse matrix. This will take a while.

In [4]:
n_users = df.userid.unique().shape[0]
n_items = df.targetid.unique().shape[0]
assert n_users == len(user_map)
assert n_items == len(item_map)

print("{} users.".format(n_users))
print("{} items.".format(n_items))

38903 users.
857382 items.


In [5]:
ratings = scipy.sparse.csr_matrix((df['rating'], (df.userid.map(user_map), df.targetid.map(item_map))),
                                  shape=(n_users, n_items))

The function below was adapted from the blog post linked above. See: http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/#Collaborative-filtering

I've made two changes:
 * I no longer use epsilon because scipy dies when I try to add a scalar to a sparse matrix. This shouldn't be matter because all users in the matrix have at least one favorite.
 * I use `sim.diagonal()` instead of `np.diag()` because sparse matrices complain mightily when it comes to `np.diag()`
 * Things die if we try to do regular division. Instead, we construct a diagonal matrix with the
   reciprocals of everything we would have divided with and multiple
Not sure if this is fully correct.

In [54]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T)
    elif kind == 'item':
        sim = ratings.T.dot(ratings)
    norms = np.array([np.sqrt(sim.diagonal())])
    norms_sparse_diag = scipy.sparse.diags(1/norms.ravel(), format='csr')
    return (norms_sparse_diag * sim * norms_sparse_diag)

In [15]:
sim = ratings.dot(ratings.T)

In [38]:
norms = np.array([np.sqrt(sim.diagonal())])
print("Norms made.")
norms_sparse = scipy.sparse.csr_matrix(norms)

Norms made.


In [51]:
print sim.shape
print norms.shape
print norms_sparse.shape
print norms.T.shape

print norms
print 1/norms
trick = scipy.sparse.diags(1/norms.ravel(), format='csr')
foo = trick * sim
print sim
print foo

(38903, 38903)
(1, 38903)
(1, 38903)
(38903, 1)
[[ 14.96662955   4.47213595  26.38181192 ...,   2.           4.           2.        ]]
[[ 0.06681531  0.2236068   0.0379049  ...,  0.5         0.25        0.5       ]]
  (0, 0)	224
  (0, 1)	4
  (0, 2)	8
  (0, 3)	4
  (0, 9)	4
  (0, 13)	4
  (0, 14)	4
  (0, 15)	4
  (0, 18)	4
  (0, 25)	8
  (0, 27)	4
  (0, 41)	4
  (0, 42)	4
  (0, 43)	4
  (0, 46)	4
  (0, 50)	4
  (0, 51)	4
  (0, 54)	4
  (0, 73)	4
  (0, 78)	4
  (0, 86)	8
  (0, 87)	8
  (0, 89)	4
  (0, 92)	4
  (0, 114)	4
  :	:
  (38901, 11875)	4
  (38901, 12383)	4
  (38901, 12896)	4
  (38901, 13837)	4
  (38901, 14019)	4
  (38901, 14228)	8
  (38901, 15189)	4
  (38901, 17607)	4
  (38901, 19075)	4
  (38901, 19255)	4
  (38901, 20527)	4
  (38901, 22152)	8
  (38901, 22447)	4
  (38901, 24724)	4
  (38901, 24847)	12
  (38901, 25460)	4
  (38901, 26380)	4
  (38901, 27300)	4
  (38901, 28773)	4
  (38901, 30862)	4
  (38901, 30943)	4
  (38901, 31731)	4
  (38901, 33820)	4
  (38901, 38901)	16
  (38902, 38902)	4
  (

In [57]:
user_similarity[0,1]

0.059761430466719681

In [55]:
# Now calculate user-user similarities. This will take a while during which time your computer may be unresponsive.
before = time.time()
user_similarity = fast_similarity(ratings, kind='user')
print("Calculated user similarities in {} seconds".format(time.time() - before))

Calculated user similarities in 3.51231694221 seconds


We now have user-user similarities (e.g. the cosine similarity between all pairs of users). Use them to generate recommendations.

In [ ]:
def predict_fast_simple(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
# With our user similarities in hand, let's calculate the predictions.
# You should expect this to run for a long time as well.

# TODO(hyena): There's no way this will work because it does calculate affinity for EVERYTHING. That matrix is too big.
# The solution is probably to only use the top k-closest friends.
# Until then this is disabled.

before = time.time()
#predictions = predict_fast_simple(ratings, user_similarity, kind='user')
print("Calculated predictions in {} seconds.".format(time.time() - before))

In [11]:
# Let's calculate predictions for ONE user.
# This would be faster if we didn't use a for-loop.
SKYLER = user_map[2402]  # userid for `skylerbunny`.
before = time.time()
skyler_preds = np.zeros(ratings.shape[1])

# This operation will be faster if we just use the k-closest friends.
# Calculate them:
k = 20

# This is my attempt to get just the k highest friends. As you can see in the output, it's a disaster.
top_k_friends = [np.argsort(user_similarity[SKYLER, :])[:-k-1:-1]]
print top_k_friends[0].shape[1]
print user_similarity.shape[0]
print user_similarity.shape[1]
print user_similarity[SKYLER, 30948]
print user_similarity[30948, SKYLER]
#for i in xrange(ratings.shape[1]):
#    skyler_preds[i] = (user_similarity[SKYLER, :].dot(ratings[:, i])
#                    / np.sum(np.abs(user_similarity[SKYLER, :])))
#print("Calculated recommendations for Skylerbunny in {} seconds"
#      .format(time.time() - before))

38903
38903
38903
0.0547175655165
0.0547175655165


What we'd like to do at this point is some calculation like the following pseudo code:
To calculate predictions for a given user, `i`:

Look at the top `k` most similar users to `i`. Say that user `j` is amongst them and has similarity `sim_{ij}`. Then do a weighted average of all ratings for all `k` users where `j`'s ratings are weighted by `sim_{ij}`.

Here's pseudocode showing a naive way this could be done with for loops and python dicts. Assume that we have:
 * `k_closest` is an array mapping userid keys to float similarity values
 * `score` is a list of dicts. `score[k][j]` is the score user `k` assigned to item `j`. There are no entries for unrated items.

```
scores = {}  # This will contain our predictions.

for neighbor in k_closest:
    for item, rating in score[neighbor].iteritems():
        scores[item] = scores.get(item, 0) + rating * k_closest[neighbor]
return sorted(scores, key=lambda item: -scores[item])  # Put the item with the highest score first.
```

You can see some of this idea [here](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/#Top-$k$-Collaborative-Filtering)

However, when I tried to do this, argsort wasn't behaving at all right.